In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


model_path = r"C:\Users\adhir\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0\snapshots\fe8a4ea1ffedaf415f4da2f062534de366a451e6"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
tokenizer.pad_token = tokenizer.eos_token  
tokenizer.padding_side = "right"


In [2]:
from datasets import load_dataset
tokenizer.pad_token = tokenizer.eos_token
def preprocess_batch(batch):
    columns_to_process = ["X", "season", "date", "home_team", "away_team", "home_clearances",
                          "home_corners", "home_fouls_conceded", "home_offsides", "home_passes",
                          "home_possession", "home_red_cards", "home_shots", "home_shots_on_target",
                          "home_tackles", "home_touches", "home_yellow_cards", "away_clearances",
                          "away_corners", "away_fouls_conceded", "away_offsides", "away_passes",
                          "away_possession", "away_red_cards", "away_shots", "away_shots_on_target",
                          "away_tackles", "away_touches", "away_yellow_cards", "goal_home_ft",
                          "goal_away_ft", "sg_match_ft", "result"]


    text_inputs = []
    batch_size = len(batch[columns_to_process[0]]) # Get batch size from any column

    for i in range(batch_size):
        example_text = " ".join([f"{col}: {str(batch[col][i])}" for col in columns_to_process])
        text_inputs.append(example_text)
    
   # print("----------------------")
   # print(text_inputs[0])
   # print(tokenizer(text_inputs, truncation=True, padding=True, max_length=200))
    return tokenizer(text_inputs, truncation=True, padding=True, max_length=200)

raw_data = load_dataset("WideMan/football_matches", split="train[3070:4070]")
data = raw_data.train_test_split(train_size=0.95)

tokenized_dataset = data.map(
    preprocess_batch,
    batched=True,
    batch_size=4,
    remove_columns=data["train"].column_names,
)

# Print dataset details
print(tokenized_dataset)


Map:   0%|          | 0/950 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 950
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 50
    })
})


In [3]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
data_collator

DataCollatorForLanguageModeling(tokenizer=LlamaTokenizerFast(name_or_path='C:\Users\adhir\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0\snapshots\fe8a4ea1ffedaf415f4da2f062534de366a451e6', vocab_size=32000, model_max_length=2048, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
), mlm=False, mlm_probability=0.15, mask_replace_prob=0.8, random_replace_prob=0.1, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt', seed=None)

In [6]:
from peft import get_peft_model, LoraConfig, TaskType
# Define LoRA Configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
model.train()
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
output_dir="./output",
#evaluation_strategy="epoch",
save_strategy="epoch",
save_steps=500,
learning_rate=6e-5,
weight_decay=0.01,
num_train_epochs=3,
per_device_train_batch_size=2,    # Batch size (adjust for GPU memory)
per_device_eval_batch_size=2,
logging_steps=50,
logging_dir="./logs",
resume_from_checkpoint=True
)
trainer = Trainer(
    model = model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=training_args,
    data_collator=data_collator
)
trainer.train()

Using device: cuda


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
50,0.379700
100,0.375000
150,0.371600
200,0.369700
250,0.364800
300,0.365100
350,0.363700
400,0.364300
450,0.361100
500,0.362200


TrainOutput(global_step=1425, training_loss=0.3569501394974558, metrics={'train_runtime': 663.8177, 'train_samples_per_second': 4.293, 'train_steps_per_second': 2.147, 'total_flos': 3541884641280000.0, 'train_loss': 0.3569501394974558, 'epoch': 3.0})

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM
trainer.save_model(r"E:\Fine Tune\fine_tuned_tinyllama")
tokenizer.save_pretrained(r"E:\Fine Tune\fine_tuned_tinyllama")

('E:\\Fine Tune\\fine_tuned_tinyllama\\tokenizer_config.json',
 'E:\\Fine Tune\\fine_tuned_tinyllama\\special_tokens_map.json',
 'E:\\Fine Tune\\fine_tuned_tinyllama\\tokenizer.json')

In [4]:
from peft import PeftModel, PeftConfig
config = PeftConfig.from_pretrained(r"E:\Fine Tune\fine_tuned_tinyllama")
base_model = AutoModelForCausalLM.from_pretrained(r"C:\Users\adhir\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0\snapshots\fe8a4ea1ffedaf415f4da2f062534de366a451e6")
model = PeftModel.from_pretrained(base_model, r"E:\Fine Tune\fine_tuned_tinyllama")
model


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear(in_featu

In [21]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Ensure CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load base model and tokenizer
base_model = AutoModelForCausalLM.from_pretrained(
    r"C:\Users\adhir\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0\snapshots\fe8a4ea1ffedaf415f4da2f062534de366a451e6"
).to(device)

tokenizer = AutoTokenizer.from_pretrained(
    r"C:\Users\adhir\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0\snapshots\fe8a4ea1ffedaf415f4da2f062534de366a451e6"
)

# Load fine-tuned LoRA model and move to GPU
model_with_lora = PeftModel.from_pretrained(
    base_model, r"E:\Fine Tune\fine_tuned_tinyllama"
).to(device)

prompt = "<|system|>\nYou are a helpful assistant.\n<|user|>\nwhat is the number of home_touches and away_touches in Liverpool vs Manchester City on July 2, 2021.\n<|assistant|>\n"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

lora_outputs = model_with_lora.generate(
    **inputs,
    max_new_tokens=300,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

print("LoRA fine-tuned model:", tokenizer.decode(lora_outputs[0], skip_special_tokens=True))
with torch.no_grad():
    base_outputs = base_model.generate(
        **inputs,
        max_new_tokens=300,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

print("Base model:", tokenizer.decode(base_outputs[0], skip_special_tokens=True))


Using device: cuda
LoRA fine-tuned model: <|system|>
You are a helpful assistant.
<|user|>
what is the number of home_touches and away_touches in Liverpool vs Manchester City on July 2, 2021.
<|assistant|>
The number of home_touches and away_touches in Liverpool vs Manchester City on July 2, 2021 is:
home_touches: 25
away_touches: 32
home_touches_per_game: 2.92
away_touches_per_game: 4.06
home_shots_on_target: 2
away_shots_on_target: 2
home_corner_fouls: 2
away_corner_fouls: 0
home_fouls_conceded: 3
away_fouls_conceded: 2
home_offenses_conceded: 1
away_offenses_conceded: 0
home_red_cards: 0
away_red_cards: 0
home_yellow_cards: 0
away_yellow_cards: 1
home_possession: 45
away_possession: 52
home_saves: 1
away_saves: 0
home_shutouts: 1
away_shutouts: 0
home_goals_scored: 1
away_goals_scored: 0
home_goals_conceded: 0
away_
Base model: <|system|>
You are a helpful assistant.
<|user|>
what is the number of home_touches and away_touches in Liverpool vs Manchester City on July 2, 2021.
<|assis